# Reconstruction
This is a custom reconstruction pipeline using SuperGlue and SuperPoint (SuperPoint for local feature detection and description and SuperPoint for robust Matching)
https://github.com/magicleap/SuperGluePretrainedNetwork/tree/master?tab=readme-ov-file

The first steps (detection, description and matching are done by SuperPoint + SuperGlue), the Reconstruction itself is done by COLMAP.

This reconstruction pipeline only works if there is a COLMAP reconstruction already in existence. This reconstruciton will be cloned and the custom pipeline will reconstruct it again. This is because the pipeline was mainly used for evaluating this pipeline to a normal COLMAP pipeline!

In [1]:
import os, time, sys
from pathlib import Path
import numpy as np

ROOT = Path().absolute().parent
SESSION_NAME = "test_colmap_project"
SESSION_ID = SESSION_NAME if SESSION_NAME else str(int(time.time()))
INPUT_IMAGES = f"{ROOT}/data/2_input" # initial input images

if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))

SCRIPTS_PATH = f"{ROOT}/third_party/SuperGluePretrainedNetwork"
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

COLMAP_PY_SCRIPTS_PATH = f"{ROOT}/third_party/colmap/scripts/python"
if COLMAP_PY_SCRIPTS_PATH not in sys.path:
    sys.path.append(COLMAP_PY_SCRIPTS_PATH)

RESULTS_PATH = Path(f"{ROOT}/data/3_results")
SESSION_PATH = Path(f"{RESULTS_PATH}/{SESSION_ID}")
OUT_DIR = f"{SESSION_PATH}/SuperGlue_dump"
MATCH_SCRIPT_DIR = f"{SCRIPTS_PATH}/match_pairs.py"

if not os.path.exists(SESSION_PATH):
    !mkdir -p {SESSION_PATH}
    !mkdir -p {OUT_DIR}
    print(f"Created new session with ID {SESSION_ID} under {RESULTS_PATH}")

Created new session with ID test_colmap_project under /mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results


In [2]:
# create COLMAP project and initialize database
! bash run_colmap_setup.sh "{SESSION_PATH}" "{INPUT_IMAGES}"

# set input images folder to self contained image folder of workspace/project/session path
#INPUT_IMAGES = f"{SESSION_PATH}/image_path"

I0327 20:18:27.731353  2582 misc.cc:198] 
Feature extraction
I0327 20:18:28.226994  2600 feature_extraction.cc:254] Processed file [1/27]
I0327 20:18:28.227023  2600 feature_extraction.cc:257]   Name:            IMG_0712.png
I0327 20:18:28.227030  2600 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0327 20:18:28.227033  2600 feature_extraction.cc:286]   Camera:          #1 - SIMPLE_RADIAL
I0327 20:18:28.227039  2600 feature_extraction.cc:289]   Focal Length:    1965.60px
I0327 20:18:28.227048  2600 feature_extraction.cc:302]   Features:        11514
I0327 20:18:28.441990  2600 feature_extraction.cc:254] Processed file [2/27]
I0327 20:18:28.442021  2600 feature_extraction.cc:257]   Name:            IMG_0713.png
I0327 20:18:28.442026  2600 feature_extraction.cc:283]   Dimensions:      1228 x 1638
I0327 20:18:28.442030  2600 feature_extraction.cc:286]   Camera:          #2 - SIMPLE_RADIAL
I0327 20:18:28.442034  2600 feature_extraction.cc:289]   Focal Length:    1965.60px
I0327

In [3]:
# Establish DB Connection
from database import COLMAPDatabase, pair_id_to_image_ids, image_ids_to_pair_id
import sqlite3

# create db here
DB_PATH = f"{SESSION_PATH}/database.db"
colmap_db = COLMAPDatabase.connect(DB_PATH)
cursor = colmap_db.cursor()

cursor.execute("SELECT name FROM sqlite_master WHERE type='table'")
print(cursor.fetchall())

[('cameras',), ('sqlite_sequence',), ('images',), ('keypoints',), ('descriptors',), ('matches',), ('two_view_geometries',)]


# Start of Custom Pipeline
STEPS
1. SERIALIZE: generate matching_images text file for SuperGlue to use and know which images to match
2. PROCESS: match images with SuperGlue
3. DESERIALIZE: collect information on matches for each image pair and deserialize image pairs from the generated files generically
4. POST-PROCESS/ESTIMATE: estimate which matches should be used based on "match_confidence"
5. DB OVERWRITE: Loop through all DB entries in images, matches, two_view_geometries, keypoints, ?descriptors? and overwrite the entries
6. Run COLMAP rest of steps after matching for sparse reconstruction

In [4]:
# prepare data to be read and replaced
def name_part(full_name):
    split_name = full_name.split(".")
    if len(split_name) == 0:
        return full_name
    return split_name[0]

def get_image_id_from_name(cursor, img_name):
    name_only = name_part(img_name)
    cursor.execute(f"SELECT image_id FROM images WHERE name LIKE '{name_only}%'")
    image_data = cursor.fetchone()
    if not image_data: return None
    return image_data[0]

def get_image_name_from_id(cursor, img_id):
    cursor.execute(f"SELECT name FROM images WHERE image_id={img_id};")
    image_data = cursor.fetchone()
    if not image_data: return None
    img_name, = image_data
    return img_name

# generate list of image pairs used for matching
def incremental_matching_pair_finder(image_ids):
    # first: sort all ids
    sorted_image_ids = sorted(image_ids)
    pairs = []
    for i in range(len(sorted_image_ids)):
        for j in range(i + 1, len(sorted_image_ids)):
            pairs.append((sorted_image_ids[i], sorted_image_ids[j]))
            #print(f"Pairing Images {sorted_image_ids[i]} / {sorted_image_ids[j]}")
    return pairs

# create mapping of images to be matched in a format where it can be stored in a text file
def generate_matching_map(cursor):
    image_ids = [image_id for image_id, in cursor.execute("SELECT image_id FROM images").fetchall()]
    image_pairs = incremental_matching_pair_finder(image_ids)
    lines = []
    for img1, img2 in image_pairs:
        img1 = int(img1)
        img2 = int(img2)
        img1_name = get_image_name_from_id(cursor, img1)
        img2_name = get_image_name_from_id(cursor, img2)
        line = f"{img1_name} {img2_name}"
        lines.append(line)
        #print(f"Matching {img1} / {img2}")
        #print(f"{img1_name} {img2_name} OTHER PARAMS HERE")
    return lines

# write image pairs to be matched in a text file
def write_superglue_image_pairs_file(image_pairs, file_name):
    with open(file_name, "w+") as f:
        f.writelines(f"{line}\n" for line in image_pairs)
    return

IMAGE_PAIR_FILE_NAME = "image_pairs.txt"
IMAGE_PAIR_FILE_PATH = Path(f"{SESSION_PATH}/{IMAGE_PAIR_FILE_NAME}")
image_pairs = generate_matching_map(cursor)
write_superglue_image_pairs_file(image_pairs, IMAGE_PAIR_FILE_PATH)

In [5]:
# execute SuperGlue

! python "{MATCH_SCRIPT_DIR}" \
    --input_pairs "{IMAGE_PAIR_FILE_PATH}" \
    --input_dir "{INPUT_IMAGES}" \
    --output_dir "{OUT_DIR}" \
    --resize "-1" \
    --superglue outdoor \
    --max_keypoints "-1" \
#    --viz \
#    --eval # requires some evaluation rows??

Namespace(input_pairs='/mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results/test_colmap_project/image_pairs.txt', input_dir='/mnt/d/dev/python/historical-photo-sfm-pipeline/data/2_input', output_dir='/mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results/test_colmap_project/SuperGlue_dump', max_length=-1, resize=[-1], resize_float=False, superglue='outdoor', max_keypoints=-1, keypoint_threshold=0.005, nms_radius=4, sinkhorn_iterations=20, match_threshold=0.2, viz=False, eval=False, fast_viz=False, cache=False, show_keypoints=False, viz_extension='png', opencv_display=False, shuffle=False, force_cpu=False)
Will not resize images
Running inference on device "cuda"
Loaded SuperPoint model
Loaded SuperGlue model ("outdoor" weights)
Looking for data in directory "/mnt/d/dev/python/historical-photo-sfm-pipeline/data/2_input"
Will write matches to directory "/mnt/d/dev/python/historical-photo-sfm-pipeline/data/3_results/test_colmap_project/SuperGlue_dump"
[Finished pair     

In [6]:
# read and structure dumps

def get_npz_info(path):
    npz = np.load(path)
    #print(npz.files)
    image_pairs = os.path.basename(path).split("_")
    img1_name = image_pairs[0] + "_" + image_pairs[1]
    img2_name = image_pairs[2] + "_" + image_pairs[3]
    return np.load(path), (img1_name, img2_name)
    
def read_superglue_dumps(dumps_directory):
    deserialized_dumps = []
    npz_files = []
    for file in os.listdir(dumps_directory):
        if file.endswith(".npz"):
            abs_file_name = os.path.join(dumps_directory, file)
            npz_files.append(abs_file_name)

    print(f"Found {len(npz_files)} .npz SuperGlue dump files")
    for npz_file in npz_files:
        data, (img1_name, img2_name) = get_npz_info(npz_file)
        print(img1_name, img2_name)
        deserialized_dumps.append({
            "img1_name": img1_name,
            "img2_name": img2_name,
            "img1_id": get_image_id_from_name(cursor, img1_name),
            "img2_id": get_image_id_from_name(cursor, img2_name),
            "npz_data": data
        })
    return deserialized_dumps

def get_kp_info(kp_id, npz):
    if kp_id >= npz['keypoints0'].shape[0]: return
    kp = npz['keypoints0'][kp_id]
    print(f"KP coordinates for kp ID {kp_id} = (x {int(kp[0])}, y {int(kp[1])})")
    if npz['matches'][kp_id] == -1:
        print("No Match!")
    else:
        print(f"Match {npz['matches'][kp_id]}! Confidence = {npz['match_confidence'][kp_id]}")

"""
npz, _ = get_npz_info(f'{OUT_DIR}/IMG_1140_IMG_1141_matches.npz')
print(npz['keypoints0'].shape) # kps image 1
print(npz['keypoints1'].shape) # kps image 2
kp_id_image1 = 5000 # id of the keypoint
print(npz['keypoints0'][kp_id_image1])
print(npz['matches'].shape)
print(npz['matches'][kp_id_image1])
print(npz['match_confidence'].shape)
print(npz['match_confidence'][kp_id_image1])

for i in range(1, 10):
    get_kp_info(i, npz)
"""

superglue_dumps = read_superglue_dumps(OUT_DIR)

Found 351 .npz SuperGlue dump files
IMG_0712 IMG_0713
IMG_0712 IMG_0714
IMG_0712 IMG_0715
IMG_0712 IMG_0716
IMG_0712 IMG_0717
IMG_0712 IMG_0718
IMG_0712 IMG_0719
IMG_0712 IMG_0720
IMG_0712 IMG_0721
IMG_0712 IMG_0722
IMG_0712 IMG_0723
IMG_0712 IMG_0724
IMG_0712 IMG_0725
IMG_0712 IMG_0726
IMG_0712 IMG_0727
IMG_0712 IMG_0728
IMG_0712 IMG_0729
IMG_0712 IMG_0730
IMG_0712 IMG_0731
IMG_0712 IMG_0732
IMG_0712 IMG_0733
IMG_0712 IMG_0734
IMG_0712 IMG_0735
IMG_0712 IMG_0736
IMG_0712 IMG_0737
IMG_0712 IMG_0738
IMG_0713 IMG_0714
IMG_0713 IMG_0715
IMG_0713 IMG_0716
IMG_0713 IMG_0717
IMG_0713 IMG_0718
IMG_0713 IMG_0719
IMG_0713 IMG_0720
IMG_0713 IMG_0721
IMG_0713 IMG_0722
IMG_0713 IMG_0723
IMG_0713 IMG_0724
IMG_0713 IMG_0725
IMG_0713 IMG_0726
IMG_0713 IMG_0727
IMG_0713 IMG_0728
IMG_0713 IMG_0729
IMG_0713 IMG_0730
IMG_0713 IMG_0731
IMG_0713 IMG_0732
IMG_0713 IMG_0733
IMG_0713 IMG_0734
IMG_0713 IMG_0735
IMG_0713 IMG_0736
IMG_0713 IMG_0737
IMG_0713 IMG_0738
IMG_0714 IMG_0715
IMG_0714 IMG_0716
IMG_0714 I

In [7]:
def get_matches(dump, min_confidence=0.3):
    # For each keypoint in keypoints0, the matches array indicates the index of the matching keypoint in keypoints1, or -1 if the keypoint is unmatched.
    kp_matching_pairs = []
    match_confidence = []
    kp_coordinates = []
    data = dump['npz_data']
    for i, match in enumerate(data['matches']):
        if match == -1:
            continue
        if data['match_confidence'][i] < min_confidence:
            continue
        # 0 = keypoints0 index, 1 = keypoints1 match pair if not -1
        kp_matching_pairs.append(np.array([i, match]))
        match_confidence.append(data['match_confidence'][i])
        kp_coordinates.append(np.array([data['keypoints0'][i], data['keypoints1'][match]]))

    has_items = len(kp_matching_pairs) > 0
    if has_items:
        return np.asarray(kp_matching_pairs), np.asarray(match_confidence), np.asarray(kp_coordinates)
    return np.empty([0, 2]), np.empty([0]), np.empty([0, 2])


def run_geometric_verification(cursor):
    #for pair_id, data in cursor.execute("SELECT pair_id, data FROM two_view_geometries"): return
    return

def synchronize_db_with_dumps(cursor, dumps, validate=True):
    num_db_matched_image_pairs = cursor.execute("SELECT COUNT(*) FROM matches").fetchone()[0]
    validation_errors = []

    # validate number of keypoints matches for all reoccuring images!
    keypoint_map = {}
    for d in dumps:
        npz_data = d['npz_data']
        img1_id = d['img1_id']
        img2_id = d['img2_id']
        img1_kp_num = npz_data['keypoints0'].shape[0]
        img2_kp_num = npz_data['keypoints1'].shape[0]
        if img1_id not in keypoint_map:
            keypoint_map[img1_id] = {
                "num_kps": img1_kp_num,
                "data": {
                    "keypoints": npz_data['keypoints0'],
                    "matches": npz_data['matches'],
                    "match_confidence": npz_data['match_confidence']
                }
            }
        elif keypoint_map[img1_id]['num_kps'] != img1_kp_num:
            validation_errors.append(f"ERROR: keypoint mismatch for image {d['img1_name']}: {keypoint_map[img1_id]['num_kps']} != {img1_kp_num}")

        if img2_id not in keypoint_map:
            keypoint_map[img2_id] = {
                "num_kps": img2_kp_num,
                "data": {
                    "keypoints": npz_data['keypoints1'],
                    "matches": npz_data['matches'],
                    "match_confidence": npz_data['match_confidence']
                }
            }
        elif keypoint_map[img2_id]['num_kps'] != img2_kp_num:
            validation_errors.append(f"ERROR: keypoint mismatch for image {d['img2_name']}: {keypoint_map[img2_id]['num_kps']} != {img2_kp_num}")
    
    if validate and len(validation_errors) > 0:
        print("WARNING! Validation errors:")
        for i, v_error in enumerate(validation_errors):
            print(f"{i+1}.\t {v_error}")
        #print("Aborting synchronization due to errors")
        #return

    # clear all existing information
    # descriptors can be deleted forever, because they will not be used in further reconstruction!
    cursor.execute("DELETE FROM descriptors")
    cursor.execute("DELETE FROM keypoints")
    cursor.execute("DELETE FROM matches")
    cursor.execute("DELETE FROM two_view_geometries")
    #colmap_db.commit()
    
    # write keypoints
    for image_id, img_info in keypoint_map.items():
        data = img_info['data']
        colmap_db.add_keypoints(image_id, data['keypoints'])

    # write matches and inlier matches
    dumps_length = len(dumps)
    for i, d in enumerate(dumps):
        m_idx, m_c, kp_coords = get_matches(d)
        colmap_db.add_matches(d['img1_id'], d['img2_id'], m_idx)
        tv_m_idx, tv_m_c, tv_kp_coords = get_matches(d, 0.45)
        colmap_db.add_two_view_geometry(d['img1_id'], d['img2_id'], tv_m_idx)
        print(f"[{i+1} / {dumps_length}] | Matches {d['img1_id']} / {d['img2_id']}: {m_idx.shape} >> {tv_m_idx.shape}")
    
    # commit changes once everything worked out :)
    colmap_db.commit()
    return

synchronize_db_with_dumps(cursor, superglue_dumps, validate=True)

[1 / 351] | Matches 1 / 2: (929, 2) >> (868, 2)
[2 / 351] | Matches 1 / 3: (892, 2) >> (843, 2)
[3 / 351] | Matches 1 / 4: (558, 2) >> (489, 2)
[4 / 351] | Matches 1 / 5: (489, 2) >> (442, 2)
[5 / 351] | Matches 1 / 6: (492, 2) >> (432, 2)
[6 / 351] | Matches 1 / 7: (418, 2) >> (366, 2)
[7 / 351] | Matches 1 / 8: (480, 2) >> (422, 2)
[8 / 351] | Matches 1 / 9: (445, 2) >> (401, 2)
[9 / 351] | Matches 1 / 10: (492, 2) >> (450, 2)
[10 / 351] | Matches 1 / 11: (438, 2) >> (381, 2)
[11 / 351] | Matches 1 / 12: (410, 2) >> (356, 2)
[12 / 351] | Matches 1 / 13: (425, 2) >> (372, 2)
[13 / 351] | Matches 1 / 14: (392, 2) >> (334, 2)
[14 / 351] | Matches 1 / 15: (333, 2) >> (277, 2)
[15 / 351] | Matches 1 / 16: (296, 2) >> (240, 2)
[16 / 351] | Matches 1 / 17: (294, 2) >> (224, 2)
[17 / 351] | Matches 1 / 18: (160, 2) >> (104, 2)
[18 / 351] | Matches 1 / 19: (66, 2) >> (22, 2)
[19 / 351] | Matches 1 / 20: (50, 2) >> (7, 2)
[20 / 351] | Matches 1 / 21: (22, 2) >> (4, 2)
[21 / 351] | Matches 1 / 

In [8]:
colmap_db.close()

In [9]:
# Sparse Reconstruction (Step 1)
! bash run_colmap_sparse_superglue.sh "{SESSION_PATH}" "{INPUT_IMAGES}"

I0327 20:47:59.549623  2678 misc.cc:198] 
Loading database
I0327 20:47:59.572958  2678 database_cache.cc:54] Loading cameras...
I0327 20:47:59.575001  2678 database_cache.cc:64]  27 in 0.002s
I0327 20:47:59.575032  2678 database_cache.cc:72] Loading matches...
I0327 20:47:59.632382  2678 database_cache.cc:78]  350 in 0.057s
I0327 20:47:59.632422  2678 database_cache.cc:94] Loading images...
I0327 20:47:59.666473  2678 database_cache.cc:143]  27 in 0.034s (connected 27)
I0327 20:47:59.666520  2678 database_cache.cc:154] Building correspondence graph...
I0327 20:47:59.686758  2678 database_cache.cc:190]  in 0.020s (ignored 54)
I0327 20:47:59.686895  2678 timer.cc:91] Elapsed time: 0.002 [minutes]
I0327 20:47:59.694900  2678 misc.cc:198] 
Finding good initial image pair
I0327 20:47:59.882954  2678 misc.cc:198] 
Initializing with image pair #16 and #19
I0327 20:47:59.886901  2678 misc.cc:198] 
Global bundle adjustment
iter      cost      cost_change  |gradient|   |step|    tr_ratio  tr_rad

In [10]:
generate_transforms_script = f"{ROOT}/scripts/third_party/neuralangelo/convert_data_to_json_advanced.py"
! python {generate_transforms_script} \
    --data_dir "{SESSION_PATH}" \
    --scene_type "outdoor" \
    --image_dir "{INPUT_IMAGES}"

/mnt/d/dev/python/historical-photo-sfm-pipeline
Fraction of images looking at the center: 0.37.
Fraction of images positioned around the center: 0.67.
Valid fraction of concentric images: 0.30.
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90E1CA850>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B132AC0>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B13B4C0>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B13B5B0>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B13B6A0>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B13B790>
No EXIF data found for image <PIL.PngImagePlugin.PngImageFile image mode=RGBA size=1228x1638 at 0x7FA90B13B880>
No EXIF data found for